# **군집화(Clustering): 개인실습용**

## **1. 라이브러리 로드**

In [3]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action="ignore")

import os

# 데이터 정규화 패키지
from sklearn.preprocessing import StandardScaler

# 군집분석 관련 패키지
from sklearn import cluster
from sklearn import mixture
from sklearn import datasets
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import MeanShift


# 차원축소를 위한 패키지
from sklearn.decomposition import PCA

# 유사도 측정 관련 패키지
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import mahalanobis
from scipy.stats import chi2

# 시각화 패키지
import matplotlib
import matplotlib.pyplot as plt

# 그래프의 style을 'seaborn-whitegrid'로 설정
plt.style.use(["seaborn-whitegrid"])

# 마이너스 기호(-) 깨짐 방지
matplotlib.rcParams["axes.unicode_minus"] = False

# 한글 폰트 사용하기
plt.rc("font", family="Malgun Gothic")

----------------------------------------------------------------------------------

## **2.다양한 형태의 군집화**

In [4]:
# 데이터를 무작위로 생성
np.random.seed(0)
n_samples = 1500
random_state = 0
noise = 0.05

In [5]:
# 2차원 평면공간에 주어진 dataset에 대한 산점도를 시각화하는 함수
def plot_data(dataset, position, title):
    X, y = dataset  # 2차원 배열로 X는 데이터 포인트에 대한 튜플, y는 label
    plt.subplot(position)  # 매개변수 position 자리에 오는 argument의 위치에 subplot을 그림
    plt.title(title)  # plot의 제목을 매개변수 title 자리에 오는 argument로 설정
    plt.scatter(X[:, 0], X[:, 1])  # 2개의 X변수를 사용하여 산점도를 시각화

In [3]:
# 다양한 형태의 군집화 결과

In [ ]:
# 결과 시각화
plt.figure(figsize=(12, 12))
plot_data(circles, 221, "Circles")
plot_data(moons, 222, "Moons")
plot_data(blobs, 223, "Blobs")
plot_data(no_structures, 224, "No structures")

In [ ]:
# argument에 주어진 모델을 사용하여 데이터셋에 대해 군집화를 수행하는 함수
def fit_predict_plot(model, dataset, position, title):
    X, y = dataset
    model.fit(X)  # 모델을 사용하여 데이터 X에 대해 학습을 수행
    if hasattr(model, "labels_"):  # 모델이 labels_ 속성을 가지고 있는지 확인
        labels = model.labels_.astype(int)
    else:
        labels = model.predict(X)

    colors = np.array(
        [
            "#30A9DE",
            "#E53A40",
            "#090707",
            "#A593E0",
            "#F6B352",
            "#519D9E",
            "#D81159",
            "#8CD790",
            "#353866",
        ]
    )
    ax = plt.subplot(position)
    ax.set_title(title)
    ax.scatter(X[:, 0], X[:, 1], color=colors[labels])

----------------------------------------------------------------------------------

## **3.데이터 셋 로드 및 전처리**
- **Secom 데이터 셋**
- **반도체 제조 공정에서 수집된 데이터**
- **591개의 X변수가 존재하며, 레이블에 양/불량을 의미하는 pass/fail이라는 클래스가 존재**
- **1567개의 관측치가 존재**

In [ ]:
# 파일 경로
data_path = "../Data/secom.data"
labels_path = "../Data/secom_labels.data"
# names_path = "../Data/secom.names"

# 1. 데이터 파일 읽기
secom_data = pd.read_csv(data_path, sep=' ', header=None)  # 공백으로 구분된 secom.data 파일 읽기
secom_labels = pd.read_csv(labels_path, sep=' ', header=None, names=['label', 'timestamp'])  # secom_labels.data 파일 읽기

# 2. 레이블 및 타임스탬프를 secom_data에 추가
secom_data['label'] = secom_labels['label']  # 레이블 정보 추가
secom_data['timestamp'] = pd.to_datetime(  # 타임스탬프 정보를 datetime 형식으로 변환 후 추가
    secom_labels['timestamp'], format='%d/%m/%Y %H:%M:%S'
)

# 3. 결측치 처리: 각 열의 결측치를 평균값으로 대체
secom_data.fillna(secom_data.mean(), inplace=True)  # 결측치를 각 열의 평균값으로 대체

# 4. 시간대별로 데이터 정렬
secom_data_sorted = secom_data.sort_values(by='timestamp')  # timestamp를 기준으로 데이터 정렬

# 5. timestamp 열을 가장 왼쪽으로 배치
columns = ['timestamp'] + [col for col in secom_data_sorted.columns if col != 'timestamp']
secom_data_sorted = secom_data_sorted[columns]

In [21]:
# 데이터 확인
secom_data_sorted

,timestamp,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,label
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.500500,...,97.934373,0.5005,0.0118,0.0035,2.3630,0.021458,0.016475,0.005283,99.670066,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.496600,...,208.204500,0.5019,0.0223,0.0055,4.4447,0.009600,0.020100,0.006000,208.204500,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.443600,...,82.860200,0.4958,0.0157,0.0039,3.1745,0.058400,0.048400,0.014800,82.860200,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.488200,...,73.843200,0.4990,0.0103,0.0025,2.0544,0.020200,0.014900,0.004400,73.843200,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.503100,...,97.934373,0.4800,0.4766,0.1045,99.3032,0.020200,0.014900,0.004400,73.843200,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2008-10-16 15:13:00,2899.41,2464.36,2179.7333,3085.3781,1.4843,100.0,82.2467,0.1248,1.342400,...,203.172000,0.4988,0.0143,0.0039,2.8669,0.006800,0.013800,0.004700,203.172000,-1
1563,2008-10-16 20:49:00,3052.31,2522.55,2198.5667,1124.6595,0.8763,100.0,98.4689,0.1205,1.433300,...,97.934373,0.4975,0.0131,0.0036,2.6238,0.006800,0.013800,0.004700,203.172000,-1
1564,2008-10-17 05:26:00,2978.81,2379.78,2206.3000,1110.4967,0.8236,100.0,99.4122,0.1208,1.462862,...,43.523100,0.4987,0.0153,0.0041,3.0590,0.019700,0.008600,0.002500,43.523100,-1
1565,2008-10-17 06:01:00,2894.92,2532.01,2177.0333,1183.7287,1.5726,100.0,98.7978,0.1213,1.462200,...,93.494100,0.5004,0.0178,0.0038,3.5662,0.026200,0.024500,0.007500,93.494100,-1


In [ ]:
# 데이터셋의 요약 정보 확인

In [ ]:
# 데이터 셋의 X,Y 변수 분리

In [ ]:
# 스케일링

----------------------------------------------------------------------------------

## **4. 군집화 기법 선택 (해당 파일에서는 강의시간에 학습한 군집화 기법 모두 적용)**

### **4-1. K-means 군집화**

In [ ]:
# K-Means 클러스터링 모델 생성 및 학습, 군집의 수 정해주기

# 클러스터 예측

In [ ]:
# PCA를 사용하여 2차원으로 축소

In [ ]:
# 시각화
plt.figure(figsize=(10, 7))

# 실제 레이블에 따른 시각화
plt.subplot(1, 2, 1)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap="viridis", s=50)
plt.title("Actual Labels")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")

# K-Means 클러스터링 결과에 따른 시각화
plt.subplot(1, 2, 2)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap="viridis", s=50)
plt.title("K-Means Clustering")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")

plt.show()

In [ ]:
# 실루엣 스코어 계산

### **4-2. Hierarchical 군집화**

In [ ]:
# AgglomerativeClustering 모델 생성 및 학습


# 예측한 클러스터 레이블 추출

In [ ]:
# 덴드로그램 생성

In [ ]:
# PCA를 사용하여 2차원으로 축소

In [ ]:
# 클러스터링 결과 시각화
plt.figure(figsize=(10, 7))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=predict, cmap="viridis")
plt.title("Agglomerative Clustering on Wine Data (PCA)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()

In [ ]:
# 실루엣 스코어 계산

### **4-3.DBSCAN (Density-Based Spatial Clustering of Applications with Noise)**

In [ ]:
# DBSCAN 클러스터링 모델 생성 (입실론 값은 자유롭게 조정)

In [ ]:
# PCA를 사용하여 2차원으로 축소

In [ ]:
# 클러스터링 결과 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(X_cancer_pca[:, 0], X_cancer_pca[:, 1], c=labels_cancer, cmap="viridis")
plt.title("DBSCAN on Breast Cancer Data")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")

In [ ]:
# 실루엣 스코어 계산

### **4-4.Mean shift**

In [10]:
# Mean Shift 클러스터링 모델 생성

In [11]:
# 클러스터링된 중심점 (centers)

In [12]:
# PCA를 사용하여 2차원으로 축소

In [13]:
# 클러스터링 결과 시각화
plt.figure(figsize=(10, 7))

# 데이터 포인트 시각화
plt.scatter(
    X_pca[:, 0], X_pca[:, 1], c=labels, cmap="viridis", marker="o", edgecolor="k", s=50
)

# 클러스터 중심점 시각화
plt.scatter(
    cluster_centers_pca[:, 0],
    cluster_centers_pca[:, 1],
    c="red",
    marker="x",
    s=200,
    label="Centers",
)

plt.title("Mean Shift Clustering on Wine Data")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.legend()
plt.show()  # 각 클러스터의 중심점이 빨간색 'x'로 표시, 밀도에 따라 자동으로 군집을 형성

In [ ]:
# 실루엣 스코어 계산